In [1]:
from transformers import AutoTokenizer, AutoConfig,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import  AutoConfig, AutoModelForCausalLM,LlamaForCausalLM,LlamaTokenizer
from tokenizers import Tokenizer
from datasets import load_dataset

2024-11-24 15:22:52.746869: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 15:22:52.761983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 15:22:52.778620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 15:22:52.783567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 15:22:52.797404: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("genehlm-merge-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer

LlamaTokenizer(name_or_path='genehlm-merge-hf', vocab_size=91597, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [3]:
model = LlamaForCausalLM.from_pretrained("genehlm-merge-hf") #continue pretrain
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(91597, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [4]:
from transformers import AutoConfig

# 加载配置
config = AutoConfig.from_pretrained('genehlm-merge-hf')

config

LlamaConfig {
  "_name_or_path": "genehlm-merge-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 91597
}

In [5]:
text='''GCTGACTCTGCCAGGATGGAATGAAATTAGGTTGTTTTAATTATAATGTAAAGTCAGTTCTAGTCAGACATAGTCACATAGGCAAGTAAGGGAACCTAAAATTGCTTGGAAT,
The primary use of LLaMA is research on large language models, including,
IATESTIYGIEYEKYPTTLEDHFGGSQRGTSLSAAAGSAVAIATGGAGLSGWYLCMYVHKEALGRLGFFGYDLQDQCGATVLSYQSDEGLALELRGPYPYAMHQKRLSAPKTYKIPRKAGKWVI'''
print("Test text:\n",text)
print(f"Tokenized by DNA-LLaMA tokenizer:{tokenizer.tokenize(text)}")

Test text:
 GCTGACTCTGCCAGGATGGAATGAAATTAGGTTGTTTTAATTATAATGTAAAGTCAGTTCTAGTCAGACATAGTCACATAGGCAAGTAAGGGAACCTAAAATTGCTTGGAAT,
The primary use of LLaMA is research on large language models, including,
IATESTIYGIEYEKYPTTLEDHFGGSQRGTSLSAAAGSAVAIATGGAGLSGWYLCMYVHKEALGRLGFFGYDLQDQCGATVLSYQSDEGLALELRGPYPYAMHQKRLSAPKTYKIPRKAGKWVI
Tokenized by DNA-LLaMA tokenizer:['▁GCTGA', 'CTCTG', 'CCAGGATG', 'GAA', 'TGAAATTA', 'G', 'GTTG', 'TTTTAATT', 'AT', 'AATG', 'TAAAGTCA', 'GT', 'TCTAGTCA', 'GAC', 'ATAG', 'TCACATA', 'GGCAA', 'GTAA', 'GGGAA', 'CCTAAAA', 'TTG', 'CTTGGAA', 'T', ',', '<0x0A>', 'The', '▁primary', '▁use', '▁of', '▁L', 'La', 'MA', '▁is', '▁research', '▁on', '▁large', '▁language', '▁models', ',', '▁including', ',', '<0x0A>', 'I', 'ATE', 'STI', 'YGI', 'EYEK', 'YP', 'TTLE', 'DH', 'FGG', 'SQRG', 'TSLS', 'AAAG', 'SAV', 'AI', 'ATGG', 'A', 'GL', 'SG', 'WY', 'LC', 'MYV', 'HK', 'EALG', 'RLG', 'FFG', 'YDLQ', 'DQ', 'CG', 'ATV', 'LSY', 'QSD', 'EG', 'LALE', 'LRGP', 'YP', 'YAM', 'HQ', 'KR', 'LSAP', 'KT', 'Y

In [6]:
import torch
from transformers import pipeline

model_id = "dnahlm-merge-hf-4g"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    #torch_dtype=torch.bfloat16, 
    device_map="auto",
)

pipe("The key to life is")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'generated_text': 'The key to life is to be a good listener. A good listener is a good friend.'}]

In [7]:
pipe("GGAATGAAATTAGGTTGTTTTAATTT")

[{'generated_text': 'GGAATGAAATTAGGTTGTTTTAATTTTTGTTGTTGTTGTTGTTG'}]

In [8]:
pipe("FIITGRSTGEEVELEKLKLARDCLMA")

[{'generated_text': 'FIITGRSTGEEVELEKLKLARDCLMAAATGTTG'}]